In [3]:
import pandas as pd
import sqlite3
import pyodbc

In [5]:
# Define the connection parameters for each database
northwind_connection = pyodbc.connect('DRIVER={SQL Server};'
                                      'SERVER=MAX-PC-DEC-2023\SQLEXPRESS;'
                                      'DATABASE=NorthWind;'
                                      'Trusted_Connection=yes;')

adventure_connection = pyodbc.connect('DRIVER={SQL Server};'
                                      'SERVER=MAX-PC-DEC-2023\SQLEXPRESS;'
                                      'DATABASE=AdventureWorks2019;'
                                      'Trusted_Connection=yes;')

aenc_connection = sqlite3.connect('/path/to/aenc.sqlite')  # Update the path to your SQLite database file

In [8]:
northwind_query = """
SELECT EmployeeID AS Id, 'NW' AS Database, FirstName, LastName, Address, City, Region, PostalCode, Country, HomePhone AS PhoneNumber, NULL AS Sex, HireDate AS StartDate, ReportsTo AS ManagerId, NULL AS TerminationDate
FROM Employees
"""

In [9]:
adventure_query = """
SELECT BusinessEntityID AS Id, 'AW' AS Database, FirstName, LastName, AddressLine1 AS Address, City, StateProvinceName AS Region, PostalCode, CountryRegionCode AS Country, PhoneNumber, Gender AS Sex, HireDate AS StartDate, NULL AS ManagerId, NULL AS TerminationDate
FROM HumanResources.Employee AS e
JOIN Person.Person AS p ON e.BusinessEntityID = p.BusinessEntityID
JOIN Person.BusinessEntityAddress AS bea ON p.BusinessEntityID = bea.BusinessEntityID
JOIN Person.Address AS a ON bea.AddressID = a.AddressID
JOIN Person.StateProvince AS sp ON a.StateProvinceID = sp.StateProvinceID
JOIN Person.PersonPhone AS pp ON p.BusinessEntityID = pp.BusinessEntityID
"""

In [10]:
aenc_query = """
SELECT e.EmployeeID AS Id, 'AENC' AS Database, e.FirstName, e.LastName, e.Address, e.City, s.StateName AS Region, e.PostalCode, e.Country, e.PhoneNumber, e.Sex, e.HireDate AS StartDate, e.ManagerID, e.TerminationDate
FROM Employee e
JOIN State s ON e.StateID = s.StateID
"""

In [15]:

# Function to execute query and return data frame
def fetch_data(query, connection):
    try:
        return pd.read_sql_query(query, connection)
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

# Fetch data from databases
northwind_data = fetch_data(northwind_query, northwind_connection)
adventure_data = fetch_data(adventure_query, adventure_connection)
aenc_data = fetch_data(aenc_query, aenc_connection)

# Combine all data into a single DataFrame
combined_data = pd.concat([northwind_data, adventure_data, aenc_data], ignore_index=True)

# Display the combined data
print(combined_data.head())

# Optional: Save the combined data to a new SQL table or a CSV file
# combined_data.to_csv('combined_employees.csv', index=False)

An error occurred: Execution failed on sql '
SELECT EmployeeID AS Id, 'NW' AS Database, FirstName, LastName, Address, City, Region, PostalCode, Country, HomePhone AS PhoneNumber, NULL AS Sex, HireDate AS StartDate, ReportsTo AS ManagerId, NULL AS TerminationDate
FROM Employees
': ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Incorrect syntax near the keyword 'Database'. (156) (SQLExecDirectW)")
An error occurred: Execution failed on sql '
SELECT BusinessEntityID AS Id, 'AW' AS Database, FirstName, LastName, AddressLine1 AS Address, City, StateProvinceName AS Region, PostalCode, CountryRegionCode AS Country, PhoneNumber, Gender AS Sex, HireDate AS StartDate, NULL AS ManagerId, NULL AS TerminationDate
FROM HumanResources.Employee AS e
JOIN Person.Person AS p ON e.BusinessEntityID = p.BusinessEntityID
JOIN Person.BusinessEntityAddress AS bea ON p.BusinessEntityID = bea.BusinessEntityID
JOIN Person.Address AS a ON bea.AddressID = a.AddressID
JOIN Person.StateProvince AS

C:\Users\Maxhb\AppData\Local\Temp\ipykernel_34052\395828943.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, connection)
C:\Users\Maxhb\AppData\Local\Temp\ipykernel_34052\395828943.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, connection)
